In [1]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
!pip install -r requirements.txt wandb

Cloning into 'yolov5'...
remote: Enumerating objects: 12398, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 12398 (delta 56), reused 44 (delta 22), pack-reused 12293
Receiving objects: 100% (12398/12398), 12.76 MiB | 25.78 MiB/s, done.
Resolving deltas: 100% (8528/8528), done.
/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 15.7 MB/s 
     |████████████████████████████████| 181 kB 80.8 MB/s 
     |████████████████████████████████| 158 kB 68.5 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 157 kB 65.1 MB/s 
     |████████████████████████████████| 157 kB 85.3 MB/s 
     |████████████████████████████████| 157 kB 81.6 MB/s 
     |████████████████████████████████| 157 kB 80.2 MB/s 
     |████████████████████████████████| 157 kB 77.3 MB/s 
     |███████████

In [2]:
from PIL import Image, ExifTags
from pycocotools.coco import COCO
from matplotlib.patches import Polygon, Rectangle
from matplotlib.collections import PatchCollection
import colorsys
import random
import pylab

import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from tqdm import tqdm

import shutil
import os

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [3]:
#%%capture
%cd ..
!gdown 1-9N7Obs67AcB-Jkucx7Ny7iaf_mKdxND
!unzip rotated.zip
%mv /content/content/* /content

/content
Downloading...
From: https://drive.google.com/uc?id=1-9N7Obs67AcB-Jkucx7Ny7iaf_mKdxND
To: /content/rotated.zip
100% 2.61G/2.61G [00:21<00:00, 123MB/s]
Archive:  rotated.zip
   creating: content/yoloTACO/images/
  inflating: content/yoloTACO/images/1311.jpg  
  inflating: content/yoloTACO/images/752.jpg  
  inflating: content/yoloTACO/images/418.jpg  
  inflating: content/yoloTACO/images/874.jpg  
  inflating: content/yoloTACO/images/1219.jpg  
  inflating: content/yoloTACO/images/1033.jpg  
  inflating: content/yoloTACO/images/67.jpg  
  inflating: content/yoloTACO/images/540.jpg  
  inflating: content/yoloTACO/images/1160.jpg  
  inflating: content/yoloTACO/images/981.jpg  
  inflating: content/yoloTACO/images/61.jpg  
  inflating: content/yoloTACO/images/157.jpg  
  inflating: content/yoloTACO/images/342.jpg  
  inflating: content/yoloTACO/images/791.jpg  
  inflating: content/yoloTACO/images/1168.jpg  
  inflating: content/yoloTACO/images/239.jpg  
  inflating: content/yolo

In [4]:
#@title
# dataset_path = './data'
# anns_file_path = dataset_path + '/' + 'annotations.json'

# # Read annotations
# with open(anns_file_path, 'r') as f:
#     dataset = json.loads(f.read())

# categories = dataset['categories']
# anns = dataset['annotations']
# imgs = dataset['images']
# nr_cats = len(categories)
# nr_annotations = len(anns)
# nr_images = len(imgs)

# # Load categories and super categories
# cat_names = []
# super_cat_names = []
# super_cat_ids = {}
# super_cat_last_name = ''
# nr_super_cats = 0
# for cat_it in categories:
#     cat_names.append(cat_it['name'])
#     super_cat_name = cat_it['supercategory']
#     # Adding new supercat
#     if super_cat_name != super_cat_last_name:
#         super_cat_names.append(super_cat_name)
#         super_cat_ids[super_cat_name] = nr_super_cats
#         super_cat_last_name = super_cat_name
#         nr_super_cats += 1

# print('Number of super categories:', nr_super_cats)
# print('Number of categories:', nr_cats)
# print('Number of annotations:', nr_annotations)
# print('Number of images:', nr_images)

In [5]:
# # Read annotations
# dataset_path = './data'
# anns_file_path = dataset_path + '/' + 'annotations.json'

# with open(anns_file_path, 'r') as f:
#     dataset = json.loads(f.read())
# imgs = dataset['images']

# data_source = COCO(annotation_file=anns_file_path)

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!


In [4]:
!pwd

/content


In [6]:
## train test split
'''
train: images/train
val: images/val
test: images/test
'''
import random
id_list=[i for i in range(1500)]
random.shuffle(id_list)

train_ids = id_list[:1050]
val_ids = id_list[1050:1050+300]
test_ids = id_list[1050+300:]

def move_helper(ids, desti):
  for id in ids:
    img_name = os.path.join( './yoloTACO/images', str(id)+'.jpg' )
    lbl_name = os.path.join( './yoloTACO/labels', str(id)+'.txt' )
    print(img_name)
    if os.path.isfile(img_name):
        shutil.move( img_name, 'yoloTACO/images/'+desti)
        shutil.move( lbl_name, 'yoloTACO/labels/'+desti)
    else :
        print('file does not exist', img_name)

In [7]:
%%capture
!mkdir yoloTACO/images/train
!mkdir yoloTACO/images/val
!mkdir yoloTACO/images/test
!mkdir yoloTACO/labels/train
!mkdir yoloTACO/labels/val
!mkdir yoloTACO/labels/test
move_helper(test_ids,'test')
move_helper(train_ids,'train')
move_helper(val_ids,'val')

In [8]:
#@title yml

with open('/content/yolov5/data/yoloTACO.yaml', mode='w') as fp:
  lines = '''path: ../yoloTACO  # dataset root dir
train: images/train  # train images (relative to 'path') 128 images
val: images/val  # val images (relative to 'path') 128 images
test: images/test # test images (optional)

# Classes
names:
  0: Aluminium foil
  1: Battery
  2: Aluminium blister pack
  3: Carded blister pack
  4: Other plastic bottle
  5: Clear plastic bottle
  6: Glass bottle
  7: Plastic bottle cap
  8: Metal bottle cap
  9: Broken glass
  10: Food Can
  11: Aerosol
  12: Drink can
  13: Toilet tube
  14: Other carton
  15: Egg carton
  16: Drink carton
  17: Corrugated carton
  18: Meal carton
  19: Pizza box
  20: Paper cup
  21: Disposable plastic cup
  22: Foam cup
  23: Glass cup
  24: Other plastic cup
  25: Food waste
  26: Glass jar
  27: Plastic lid
  28: Metal lid
  29: Other plastic
  30: Magazine paper
  31: Tissues
  32: Wrapping paper
  33: Normal paper
  34: Paper bag
  35: Plastified paper bag
  36: Plastic film
  37: Six pack rings
  38: Garbage bag
  39: Other plastic wrapper
  40: Single-use carrier bag
  41: Polypropylene bag
  42: Crisp packet
  43: Spread tub
  44: Tupperware
  45: Disposable food container
  46: Foam food container
  47: Other plastic container
  48: Plastic glooves
  49: Plastic utensils
  50: Pop tab
  51: Rope & strings
  52: Scrap metal
  53: Shoe
  54: Squeezable tube
  55: Plastic straw
  56: Paper straw
  57: Styrofoam piece
  58: Unlabeled litter
  59: Cigarette'''
  fp.writelines(lines)

In [9]:
%cd ./yolov5
!ls

/content/yolov5
benchmarks.py	 detect.py   models	       setup.cfg       val.py
classify	 export.py   README.md	       train.py
CONTRIBUTING.md  hubconf.py  requirements.txt  tutorial.ipynb
data		 LICENSE     segment	       utils


In [10]:
!nvidia-smi

Sun Sep 18 02:49:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
# check we have albu...
!pip list -v | grep [Aa]lbumentations

albumentations                1.2.1                        /usr/local/lib/python3.7/dist-packages pip


In [16]:
# Train
# run 'pip install wandb' to automatically track and visualize YOLOv5 🚀 runs in Weights & Biases
!python train.py --img 640 \
 --batch-size 16 --epochs 20 \
 --data yoloTACO.yaml \
 --weights yolov5x.pt \
 --multi-scale \
 #--freeze 10


wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 3
wandb: You chose 'Don't visualize my results'
train: weights=yolov5x.pt, cfg=, data=yoloTACO.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=True, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[7], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-128-ge8a9c5a Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (Tesla V100-SXM2-16GB, 16160MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentu

In [17]:
!python val.py --data yoloTACO.yaml --weights runs/train/exp4/weights/best.pt

val: data=/content/yolov5/data/yoloTACO.yaml, weights=['runs/train/exp3/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.2-128-ge8a9c5a Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (Tesla V100-SXM2-16GB, 16160MiB)

Fusing layers... 
Model summary: 322 layers, 86570425 parameters, 0 gradients, 205.0 GFLOPs
val: Scanning '/content/yoloTACO/labels/val.cache' images and labels... 300 found, 0 missing, 0 empty, 0 corrupt: 100% 300/300 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 10/10 [00:23<00:00,  2.36s/it]
                   all        300       1018        0.6      0.143      0.123     0.0911
Speed: 1.0ms pre-process, 11.7ms inference, 4.7ms NMS per image at shape